In [14]:
import json
import random
import torch
import torch.nn as nn
from model import BotNet
from NLP_utils import  tokenize, vectorize

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
with open('intents.json') as f:
    intents = json.load(f)
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hey :-)', 'Hello, thanks for visiting', 'Hi there, what can I do for you?', 'Hi there, how can I help?']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['Which items do you have?', 'What kinds of items are there?', 'What do you sell?'], 'responses': ['We sell coffee and tea', 'We have coffee and tea']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with Paypal?', 'Are you cash only?'], 'responses': ['We accept VISA, Mastercard and Paypal', 'We accept most major credit cards, and Paypal']}, {'tag': 'd

In [5]:
model_data = torch.load('model_data.pth')

In [6]:
model_data

{'model_state': OrderedDict([('l1.weight',
               tensor([[ 1.3122e-01,  5.0160e-01,  2.9121e-01,  2.0012e-01,  2.5023e-01,
                        -3.4425e-01,  2.6551e-01,  3.6705e-01,  3.0112e-01,  3.0404e-01,
                         2.8029e-01,  1.0778e+00,  5.8040e-01,  1.3554e+00,  9.9235e-01,
                         6.2345e-01,  1.1974e-01, -3.4496e-01,  1.9986e-01,  4.4243e-01,
                         1.6162e-01,  3.6768e-01,  1.8246e-01,  6.2506e-01,  6.0200e-01,
                         9.9741e-02,  2.9302e-01,  1.5313e+00,  1.2276e-01,  1.1695e+00,
                        -3.8056e-01,  1.1781e+00,  1.6394e-01,  1.1420e-01,  1.3406e+00,
                         6.5790e-01,  2.9523e-02,  2.6489e-01,  2.2217e-01,  3.2410e-02,
                        -3.0384e-01,  2.5107e-01,  9.5452e-01,  1.0009e+00,  7.2026e-01,
                         1.1591e+00,  3.3690e-01, -4.2684e-02,  7.8434e-02,  2.2048e-01,
                         8.3019e-01,  2.2628e-01,  2.0369e-01,  3.3

In [8]:
model_data.keys()

dict_keys(['model_state', 'input_size', 'output_size', 'hidden_size', 'total_words', 'tags'])

In [9]:
model_state = model_data['model_state']
input_size = model_data['input_size']
output_size = model_data['output_size']
hidden_size = model_data['hidden_size']
total_words = model_data['total_words']
tags = model_data['tags']

In [13]:
bot_model = BotNet(input_size, hidden_size, output_size)
bot_model.load_state_dict(model_state)
bot_model.eval()

BotNet(
  (l1): Linear(in_features=54, out_features=8, bias=True)
  (l2): Linear(in_features=8, out_features=8, bias=True)
  (l3): Linear(in_features=8, out_features=7, bias=True)
  (relu): ReLU()
)

In [23]:
bot_name = 'test_bot'
print(f'Ask anything!and type quit to exit.')

while True:
    sent = input('You: ')
    if sent == 'quit':
        break
    sent_tok = tokenize(sent)
    sent_vec = vectorize(sent_tok, total_words)
    sent_vec =sent_vec.reshape(1,-1)
    sent_vec = torch.from_numpy(sent_vec).to(device).float()
    
    out = bot_model(sent_vec)
    prob_of_out = torch.softmax(out,dim=1)
    _,pred = torch.max(out,dim=1)
    prob = prob_of_out[0][pred.item()]
    #print(prob.item(),prob_of_out[0])
    if prob.item() > 0.2:
        for intent in intents['intents']:
            if tags[pred.item()] == intent['tag']:
                print(f'{bot_name}: {random.choice(intent["responses"])}')
                
    else:
        print(f'{bot_name}: I do not understand...')
    
    
    
        

Ask anything!and type quit to exit.
You: hi
test_bot: Have a nice day
You: hi there
test_bot: What did the buffalo say when his son left for college? Bison.
You: what do you sell
test_bot: Delivery takes 2-4 days
You: thanks
test_bot: Delivery takes 2-4 days
You: who are you
test_bot: See you later, thanks for visiting
You: ok thanks
test_bot: Shipping takes 2-4 days
You: i am amit
test_bot: Bye! Come back again soon.
You: ok
test_bot: Have a nice day
You: quit
